# Laboratory Task 6


**Instruction:** Instruction: Convert the following CNN architecture diagram into a PyTorch CNN Architecture.

![Caption text](../images/quick_draw.png)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Conv1 block
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        
        # Conv2 block
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Dropout
        self.dropout = nn.Dropout(p=0.2)
        
        # Determine flatten dimension automatically
        x = torch.zeros(1, 1, 28, 28)
        x = self._forward_features(x)
        flatten_dim = x.view(1, -1).size(1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(flatten_dim, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, 10)  # MNIST → 10 classes

    def _forward_features(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.pool2(F.relu(self.conv4(x)))
        return x

    def forward(self, x):
        x = self._forward_features(x)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = F.softmax(x, dim=1)  # Apply Softmax for class probabilities
        return x

In [3]:
# Test model
model = CNNModel()
print(model)

sample = torch.randn(1, 1, 28, 28)
output = model(sample)
print("Output probabilities:", output)
print("Predicted class:", torch.argmax(output, dim=1).item())

CNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=6272, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=10, bias=True)
)
Output probabilities: tensor([[0.0992, 0.1024, 0.0959, 0.1033, 0.0999, 0.0981, 0.0987, 0.1028, 0.0993,
         0.1004]], grad_fn=<SoftmaxBackward0>)
Predicted class: 3
